# Script 27b debugging

This notebook explores different step outputs of Script 27 and function aup.pois_time() in order to find solutions to count problems.

## Import libraries

In [1]:
accesibilidad_urbana = "../../../"

In [2]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

from shapely import Point

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join(accesibilidad_urbana))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

* __Approach:__ Re-do network process in QGIS for Project_00(baseline) and project_01(plaza_italia) and re-run __without changing code.__
* __Result:__ Re-doing the network and re-running plaza italia resulted in Plaza Italia having problems. But could also be the code, not the network.

## Script 27b debugging - Restaurantes bar cafe problem

#### * __Problem:__ The restaurantes_bar_cafes_15min_count changes negatively after implementing projects.
#### * __Result:__ The problem uncovered an unecessary set() in function aup.get_seeds(). The set removed duplicate nodes, so if one node had 3 restaurantes, only 1 was counted while time remained unaffected. The set was removed.

In [3]:
script27_output_dir = accesibilidad_urbana + "data/external/santiago/output/"

### Script 27b debugging - Exploration 1: batch size creation in function pois_time()
* The problem was believed to be here. It wasn't, but still the code was changed from <__if len(x) % 250:__> to <__if (len(x) % 250) == 0:__>

In [147]:
attributes_list = ['carniceria_time','carniceria_count_15min','hogar_time','hogar_count_15min','bakeries_time','bakeries_count_15min',
                       'supermercado_time','supermercado_count_15min','banco_time','banco_count_15min','ferias_time','ferias_count_15min',
                       'local_mini_market_time','local_mini_market_count_15min','correos_time','correos_count_15min','centro_recyc_time','centro_recyc_count_15min',
                       'hospital_priv_time','hospital_priv_count_15min','hospital_pub_time','hospital_pub_count_15min','clinica_priv_time','clinica_priv_count_15min',
                       'clinica_pub_time','clinica_pub_count_15min','farmacia_time','farmacia_count_15min','vacunatorio_priv_time','vacunatorio_priv_count_15min',
                       'vacunatorio_pub_time','vacunatorio_pub_count_15min','consult_ado_priv_time','consult_ado_priv_count_15min','consult_ado_pub_time','consult_ado_pub_count_15min',
                       'salud_mental_time','salud_mental_count_15min','labs_priv_time','labs_priv_count_15min','residencia_adumayor_time','residencia_adumayor_count_15min',
                       'eq_deportivo_priv_time','eq_deportivo_priv_count_15min','eq_deportivo_pub_time','eq_deportivo_pub_count_15min','club_deportivo_time','club_deportivo_count_15min',
                       'civic_office_time','civic_office_count_15min','tax_collection_time','tax_collection_count_15min','social_security_time','social_security_count_15min',
                       'police_time','police_count_15min','bomberos_time','bomberos_count_15min','museos_priv_time','museos_priv_count_15min','museos_pub_time','museos_pub_count_15min',
                       'cines_time','cines_count_15min','sitios_historicos_time','sitios_historicos_count_15min','restaurantes_bar_cafe_time','restaurantes_bar_cafe_count_15min',
                       'librerias_time','librerias_count_15min','ep_plaza_small_time','ep_plaza_small_count_15min','ep_plaza_big_time','ep_plaza_big_count_15min',
                       'edu_basica_pub_time','edu_basica_pub_count_15min','edu_media_pub_time','edu_media_pub_count_15min','jardin_inf_pub_time','jardin_inf_pub_count_15min',
                       'universidad_time','universidad_count_15min','edu_tecnica_time','edu_tecnica_count_15min','edu_adultos_pub_time','edu_adultos_pub_count_15min',
                       'edu_especial_pub_time','edu_especial_pub_count_15min','bibliotecas_time','bibliotecas_count_15min','centro_edu_amb_time','centro_edu_amb_count_15min',
                       'paradas_tp_ruta_time','paradas_tp_ruta_count_15min','paradas_tp_metro_time','paradas_tp_metro_count_15min','paradas_tp_tren_time','paradas_tp_tren_count_15min',
                       'ciclovias_time','ciclovias_count_15min']

# 499 items
#short_list = attributes_list.copy()
#short_list.remove('ciclovias_count_15min')
#attributes_list = (attributes_list*4)
#attributes_list.extend(short_list)

# 500 items
#attributes_list = attributes_list*5

# 501 items
#attributes_list = attributes_list*5+['test']
print(len(attributes_list))

100


In [157]:
# 1486 items
extended_list = (attributes_list*15)
short_list = extended_list.copy()
remove_items = ['edu_especial_pub_time','edu_especial_pub_count_15min','bibliotecas_time','bibliotecas_count_15min','centro_edu_amb_time','centro_edu_amb_count_15min',
                'paradas_tp_ruta_time','paradas_tp_ruta_count_15min','paradas_tp_metro_time','paradas_tp_metro_count_15min','paradas_tp_tren_time','paradas_tp_tren_count_15min',
                'ciclovias_time','ciclovias_count_15min']
for item in remove_items:
    short_list.remove(item)
test_list = short_list.copy()
print(len(test_list))

1486


In [156]:
# 1490 items
extended_list = (attributes_list*15)
short_list = extended_list.copy()
remove_items = ['edu_especial_pub_time','edu_especial_pub_count_15min','bibliotecas_time','bibliotecas_count_15min','centro_edu_amb_time','centro_edu_amb_count_15min',
                'paradas_tp_ruta_time','paradas_tp_ruta_count_15min','paradas_tp_metro_time','paradas_tp_metro_count_15min']
for item in remove_items:
    short_list.remove(item)
test_list = short_list.copy()
print(len(test_list))

1490


In [154]:
# If possible, analyses by batches of 200 pois.
if (len(test_list) % 250) == 0:
    print("Escenario 1")
    batch_size = len(test_list)/200
    print(f"{int(batch_size)+1} batches")
    for k in range(int(batch_size)+1):
        
        print(f"Iteración {k}")
        source_process = test_list[int(200*k):int(200*(1+k))]
        print(f"Analysing from {int(200*k)} to {int(200*(1+k))}. ({len(source_process)} datas).")
        
else:
    print("Escenario 2")
    batch_size = len(test_list)/250
    print(f"{int(batch_size)+1} batches")
    for k in range(int(batch_size)+1):
        
        print(f"Iteración {k}")
        source_process = test_list[int(250*k):int(250*(1+k))]
        print(f"Analysing from {int(250*k)} to {int(250*(1+k))}. ({len(source_process)} datas).")

Escenario 2
6 batches
Iteración 0
Analysing from 0 to 250. (250 datas).
Iteración 1
Analysing from 250 to 500. (250 datas).
Iteración 2
Analysing from 500 to 750. (250 datas).
Iteración 3
Analysing from 750 to 1000. (250 datas).
Iteración 4
Analysing from 1000 to 1250. (250 datas).
Iteración 5
Analysing from 1250 to 1500. (240 datas).


In [158]:
# If possible, analyses by batches of 200 pois.
if len(test_list) % 250:
    batch_size = len(test_list)/201 # <----------- ADDED +1
    print(f"{int(batch_size)+1} batches")
    for k in range(int(batch_size)+1):
        
        print(f"Iteración {k}")
        source_process = test_list[int(200*k):int(200*(1+k))]
        print(f"Analysing from {int(200*k)} to {int(200*(1+k))}. ({len(source_process)} datas).")
        
else:
    batch_size = len(test_list)/251 # <----------- ADDED +1
    print(f"{int(batch_size)+1} batches")
    for k in range(int(batch_size)+1):
        
        print(f"Iteración {k}")
        source_process = test_list[int(250*k):int(250*(1+k))]
        print(f"Analysing from {int(250*k)} to {int(250*(1+k))}. ({len(source_process)} datas).")

8 batches
Iteración 0
Analysing from 0 to 200. (200 datas).
Iteración 1
Analysing from 200 to 400. (200 datas).
Iteración 2
Analysing from 400 to 600. (200 datas).
Iteración 3
Analysing from 600 to 800. (200 datas).
Iteración 4
Analysing from 800 to 1000. (200 datas).
Iteración 5
Analysing from 1000 to 1200. (200 datas).
Iteración 6
Analysing from 1200 to 1400. (200 datas).
Iteración 7
Analysing from 1400 to 1600. (86 datas).


### Script 27b debugging - Exploration 2: Restaurantes bar cafe batches
* The batches created were extracted and analysed until the set() solution was discovered.
* __In order to replicate results__ it would be necessary to save the nearest and nodes_distance_prep (batches) used inside pois_time() for each source.

#### Precalculated nearest

In [216]:
nearest_dir = accesibilidad_urbana + "data/external/santiago/nearest/"
original_nearest = gpd.read_file(nearest_dir + "nearest_restaurantes_bar_cafe.gpkg")
print(original_nearest.shape)

osmid_check_list = list(red_buena_calidad_nearest.reset_index().osmid.unique())
original_nearest = original_nearest.loc[original_nearest.osmid.isin(osmid_check_list)]

print(original_nearest.shape)
original_nearest.tail(2)

(2370, 20)
(1486, 20)


,rut,dv,vigenci,fecha,tipo_di,calle,numero,bloque,departa,villa_p,ciudad,comuna,region,sngldrs,empresa,index_right,city,osmid,distance_node,geometry
2367,99533970,6,N,2015-12-24,SUCURSAL,SANCHEZ FONTECILLA,310,None,None,None,SANTIAGO,LAS CONDES,XIII REGION METROPOLITANA,"SANCHEZ FONTECILLA 310, LAS CONDES, Chile",INVERSIONES GRS S A,0,alamedabuffer_4500m,2910188259,55.432913,POINT (-70.59839 -33.41998)
2368,99546720,8,N,2013-05-20,SUCURSAL,COMPANIA DE JESUS,2217,None,None,None,SANTIAGO,SANTIAGO,XIII REGION METROPOLITANA,"COMPANIA DE JESUS 2217, SANTIAGO, Chile",COMERCIAL BARRERA HERMANOS SPA,0,alamedabuffer_4500m,253411470,42.685830,POINT (-70.66729 -33.44010)


In [226]:
test = original_nearest.iloc[0:250]
print(test.shape)
test.tail(1)

(250, 20)


,rut,dv,vigenci,fecha,tipo_di,calle,numero,bloque,departa,villa_p,ciudad,comuna,region,sngldrs,empresa,index_right,city,osmid,distance_node,geometry
366,76152699,5,S,2016-02-02,SUCURSAL,AV PAJARITOS,4500,None,9,None,SANTIAGO,MAIPU,XIII REGION METROPOLITANA,"AV PAJARITOS 4500, MAIPU, Chile",PJ CHILE SPA,0,alamedabuffer_4500m,443979608,37.375551,POINT (-70.74829 -33.48022)


#### Red buena calidad

In [327]:
osmid = 254346788
network = 'testarea' #alamedabuffer or testarea

In [323]:
red_buena_calidad_nearest = gpd.read_file(script27_output_dir + f"red_buena_calidad_batches_{network}/restaurantes_nearest.gpkg")
red_buena_calidad_batch_0 = gpd.read_file(script27_output_dir + f"red_buena_calidad_batches_{network}/restaurantes_batch0.gpkg")
print(f"Batch 0: {len(red_buena_calidad_batch_0.loc[red_buena_calidad_batch_0.osmid==osmid])}.")

if network == 'alamedabuffer':
    red_buena_calidad_batch_1 = gpd.read_file(script27_output_dir + f"red_buena_calidad_batches_{network}/restaurantes_batch1.gpkg")
    print(f"Batch 1: {len(red_buena_calidad_batch_1.loc[red_buena_calidad_batch_1.osmid==osmid])}.")
    red_buena_calidad_batch_2 = gpd.read_file(script27_output_dir + f"red_buena_calidad_batches_{network}/restaurantes_batch2.gpkg")
    print(f"Batch 2: {len(red_buena_calidad_batch_2.loc[red_buena_calidad_batch_2.osmid==osmid])}.")
    red_buena_calidad_batch_3 = gpd.read_file(script27_output_dir + f"red_buena_calidad_batches_{network}/restaurantes_batch3.gpkg")
    print(f"Batch 3: {len(red_buena_calidad_batch_3.loc[red_buena_calidad_batch_3.osmid==osmid])}.")
    red_buena_calidad_batch_4 = gpd.read_file(script27_output_dir + f"red_buena_calidad_batches_{network}/restaurantes_batch4.gpkg")
    print(f"Batch 4: {len(red_buena_calidad_batch_4.loc[red_buena_calidad_batch_4.osmid==osmid])}.")
    red_buena_calidad_batch_5 = gpd.read_file(script27_output_dir + f"red_buena_calidad_batches_{network}/restaurantes_batch5.gpkg")
    print(f"Batch 5: {len(red_buena_calidad_batch_5.loc[red_buena_calidad_batch_5.osmid==osmid])}.")

Batch 0: 1.


In [328]:
red_buena_calidad_batch_0.loc[red_buena_calidad_batch_0.osmid==osmid]

,osmid,index,x,y,street_count,dist_restaurantes_bar_cafe,restaurantes_bar_cafe_15min,geometry
2,254346788,2,-70.674783,-33.434185,4,3.61636,5,POINT (-70.67478 -33.43419)


In [329]:
red_buena_calidad_batch_1.loc[red_buena_calidad_batch_1.osmid==osmid]

,osmid,index,x,y,street_count,dist_restaurantes_bar_cafe,restaurantes_bar_cafe_15min,geometry


In [317]:
red_buena_calidad_batch_2.loc[red_buena_calidad_batch_2.osmid==osmid]

,osmid,index,x,y,street_count,dist_restaurantes_bar_cafe,restaurantes_bar_cafe_15min,geometry
1256,254343802,1522,-70.672945,-33.435041,4,0.0,1,POINT (-70.67295 -33.43504)


In [318]:
red_buena_calidad_batch_3.loc[red_buena_calidad_batch_3.osmid==osmid]

,osmid,index,x,y,street_count,dist_restaurantes_bar_cafe,restaurantes_bar_cafe_15min,geometry
1255,254343802,1522,-70.672945,-33.435041,4,0.0,2,POINT (-70.67295 -33.43504)


In [330]:
red_buena_calidad_batch_4.loc[red_buena_calidad_batch_4.osmid==osmid]

,osmid,index,x,y,street_count,dist_restaurantes_bar_cafe,restaurantes_bar_cafe_15min,geometry


In [331]:
red_buena_calidad_batch_5.loc[red_buena_calidad_batch_5.osmid==osmid]

,osmid,index,x,y,street_count,dist_restaurantes_bar_cafe,restaurantes_bar_cafe_15min,geometry


In [304]:
nearest_in_node = red_buena_calidad_nearest.loc[red_buena_calidad_nearest.osmid==osmid]
print(len(nearest_in_node))
print(nearest_in_node.original_id.unique())

4
[1176 1468 1497 1585]


#### Norte sur

In [187]:
norte_sur_batch_0 = gpd.read_file(script27_output_dir + "norte_sur_batches/restaurantes_batch0.gpkg")
norte_sur_batch_1 = gpd.read_file(script27_output_dir + "norte_sur_batches/restaurantes_batch1.gpkg")
norte_sur_batch_2 = gpd.read_file(script27_output_dir + "norte_sur_batches/restaurantes_batch2.gpkg")
norte_sur_batch_3 = gpd.read_file(script27_output_dir + "norte_sur_batches/restaurantes_batch3.gpkg")
norte_sur_batch_4 = gpd.read_file(script27_output_dir + "norte_sur_batches/restaurantes_batch4.gpkg")
norte_sur_batch_5 = gpd.read_file(script27_output_dir + "norte_sur_batches/restaurantes_batch5.gpkg")
norte_sur_nearest = gpd.read_file(script27_output_dir + "norte_sur_batches/restaurantes_nearest.gpkg")

In [204]:
nearest_in_node = norte_sur_nearest.loc[norte_sur_nearest.osmid==osmid]
print(len(nearest_in_node))
print(nearest_in_node.original_id.unique())

4
[1176 1468 1497 1585]


In [241]:
print(len(norte_sur_batch_0.loc[norte_sur_batch_0.osmid==osmid]))
print(len(norte_sur_batch_1.loc[norte_sur_batch_1.osmid==osmid]))
print(len(norte_sur_batch_2.loc[norte_sur_batch_2.osmid==osmid]))
print(len(norte_sur_batch_3.loc[norte_sur_batch_3.osmid==osmid]))
print(len(norte_sur_batch_4.loc[norte_sur_batch_4.osmid==osmid]))
print(len(norte_sur_batch_5.loc[norte_sur_batch_5.osmid==osmid]))

0
0
0
1
0
0


In [242]:
norte_sur_batch_3.loc[norte_sur_batch_3.osmid==osmid]

,osmid,index,x,y,street_count,dist_restaurantes_bar_cafe,restaurantes_bar_cafe_15min,geometry
1290,254346790,1581,-70.673038,-33.434066,4,1.45068,2,POINT (-70.67304 -33.43407)


## Script 27b debugging - Null and ceros in data problem

#### * __Problem:__ After implementing aup.get_seeds() solution, final data of function pois_time() has NaNs in time and ceros in count.
#### * __Approach:__ Running farmacias only in red_completa only and inspect pois_time() outputs -->
#### * __Result:__ Batch analysis to all batches analysis (merge) inside pois_time() was done incorrectly. Fixed.

In [7]:
test_dir = accesibilidad_urbana + "data/external/debugging/pois_time/"

### Approach 1: Running farmacias only in red_completa only and inspect pois_time() outputs

#### __Problem 1:__ Batches add up to null in nodes_time

In [19]:
farmacias_batch0 = gpd.read_file(test_dir + "01_current_situation/nodes_distance_prep_farmacia_250batch0.gpkg")
farmacias_batch1 = gpd.read_file(test_dir + "01_current_situation/nodes_distance_prep_farmacia_250batch1.gpkg")
farmacias_batch2 = gpd.read_file(test_dir + "01_current_situation/nodes_distance_prep_farmacia_250batch2.gpkg")
farmacias_batch3 = gpd.read_file(test_dir + "01_current_situation/nodes_distance_prep_farmacia_250batch3.gpkg")

In [9]:
# Node that should be assigned to a farmacia
farmacia_osmid = 6067607409
# Node that's very close to that farmacia
near_osmid = 4764330555

In [10]:
print(f"Batch 0: {len(farmacias_batch0.loc[farmacias_batch0.osmid==farmacia_osmid])}.")
print(f"Batch 1: {len(farmacias_batch1.loc[farmacias_batch1.osmid==farmacia_osmid])}.")
print(f"Batch 2: {len(farmacias_batch2.loc[farmacias_batch2.osmid==farmacia_osmid])}.")
print(f"Batch 3: {len(farmacias_batch3.loc[farmacias_batch3.osmid==farmacia_osmid])}.")

Batch 0: 1.
Batch 1: 1.
Batch 2: 1.
Batch 3: 1.


In [25]:
farmacias_batch0.loc[farmacias_batch0.osmid==farmacia_osmid]

,osmid,index,x,y,street_count,dist_farmacia,farmacia_15min,geometry
29587,6067607409,29696,-70.712556,-33.457744,3,6.958944,3,POINT (-70.71256 -33.45774)


In [26]:
farmacias_batch1.loc[farmacias_batch1.osmid==farmacia_osmid]

,osmid,index,x,y,street_count,dist_farmacia,farmacia_15min,geometry
29587,6067607409,29696,-70.712556,-33.457744,3,14.6326,1,POINT (-70.71256 -33.45774)


In [27]:
farmacias_batch2.loc[farmacias_batch2.osmid==farmacia_osmid]

,osmid,index,x,y,street_count,dist_farmacia,farmacia_15min,geometry
29587,6067607409,29696,-70.712556,-33.457744,3,0.0,3,POINT (-70.71256 -33.45774)


In [28]:
farmacias_batch3.loc[farmacias_batch3.osmid==farmacia_osmid]

,osmid,index,x,y,street_count,dist_farmacia,farmacia_15min,geometry
29587,6067607409,29696,-70.712556,-33.457744,3,0.0,1,POINT (-70.71256 -33.45774)


In [29]:
farmacias_allbatches = gpd.read_file(test_dir + "01_current_situation/nodes_time_farmacia.gpkg")
farmacias_allbatches.loc[farmacias_allbatches.osmid==farmacia_osmid]

,osmid,x,y,street_count,time_0farmacia,farmacia_0_15min,time_1farmacia,farmacia_1_15min,time_2farmacia,farmacia_2_15min,time_3farmacia,farmacia_3_15min,time_farmacia,farmacia_15min,geometry
29696,6067607409,-70.712556,-33.457744,3,None,None,None,None,None,None,None,None,None,0.0,POINT (-70.71256 -33.45774)


In [36]:
# Problem 1 was in how batches and nodes_time was merged. Solved.
farmacias_allbatches = gpd.read_file(test_dir + "02_changing_merge/nodes_time_farmacia.gpkg")
farmacias_allbatches.loc[farmacias_allbatches.osmid==farmacia_osmid]

,osmid,x,y,street_count,time_0farmacia,farmacia_0_15min,time_1farmacia,farmacia_1_15min,time_2farmacia,farmacia_2_15min,time_3farmacia,farmacia_3_15min,time_farmacia,farmacia_15min,geometry
29696,6067607409,-70.712556,-33.457744,3,33.781947,0.0,17.6998,0.0,14.497853,1.0,11.07956,1.0,11.07956,2.0,POINT (-70.71256 -33.45774)


#### __Problem 2:__ amount of farmacia_15min is bigger on batches than it is on col.

In [81]:
farmacias_batch0 = gpd.read_file(test_dir + "02_changing_merge/nodes_distance_prep_farmacia_250batch0.gpkg")
farmacias_batch1 = gpd.read_file(test_dir + "02_changing_merge/nodes_distance_prep_farmacia_250batch1.gpkg")
farmacias_batch2 = gpd.read_file(test_dir + "02_changing_merge/nodes_distance_prep_farmacia_250batch2.gpkg")
farmacias_batch3 = gpd.read_file(test_dir + "02_changing_merge/nodes_distance_prep_farmacia_250batch3.gpkg")

In [82]:
farmacias_batch0.loc[farmacias_batch0.osmid==farmacia_osmid]

,osmid,index,x,y,street_count,dist_farmacia,farmacia_15min,geometry
29587,6067607409,29696,-70.712556,-33.457744,3,6.958944,3,POINT (-70.71256 -33.45774)


In [83]:
farmacias_batch1.loc[farmacias_batch1.osmid==farmacia_osmid]

,osmid,index,x,y,street_count,dist_farmacia,farmacia_15min,geometry
29587,6067607409,29696,-70.712556,-33.457744,3,14.6326,1,POINT (-70.71256 -33.45774)


In [84]:
farmacias_batch2.loc[farmacias_batch2.osmid==farmacia_osmid]

,osmid,index,x,y,street_count,dist_farmacia,farmacia_15min,geometry
29587,6067607409,29696,-70.712556,-33.457744,3,0.0,3,POINT (-70.71256 -33.45774)


In [85]:
farmacias_batch3.loc[farmacias_batch3.osmid==farmacia_osmid]

,osmid,index,x,y,street_count,dist_farmacia,farmacia_15min,geometry
29587,6067607409,29696,-70.712556,-33.457744,3,0.0,1,POINT (-70.71256 -33.45774)


In [86]:
# Problem 1 was in how batches and nodes_time was merged. Solved.
farmacias_allbatches = gpd.read_file(test_dir + "02_changing_merge/nodes_time_farmacia.gpkg")
farmacias_allbatches.loc[farmacias_allbatches.osmid==farmacia_osmid]

,osmid,x,y,street_count,time_0farmacia,farmacia_0_15min,time_1farmacia,farmacia_1_15min,time_2farmacia,farmacia_2_15min,time_3farmacia,farmacia_3_15min,time_farmacia,farmacia_15min,geometry
29696,6067607409,-70.712556,-33.457744,3,33.781947,0.0,17.6998,0.0,14.497853,1.0,11.07956,1.0,11.07956,2.0,POINT (-70.71256 -33.45774)


In [90]:
nodes = gpd.read_file(test_dir + "02_changing_merge/nodes_farmacia.gpkg")
# Show
nodes.head(1)

,osmid,x,y,street_count,geometry
0,386138,-70.646254,-33.442901,4,POINT (-70.64625 -33.44290)


In [91]:
empty_nodes_analysis = gpd.read_file(test_dir + "02_changing_merge/empty_nodes_analysis_farmacia.gpkg")
# Show
empty_nodes_analysis.head(1)

,osmid,x,y,street_count,geometry
0,386138,-70.646254,-33.442901,4,POINT (-70.64625 -33.44290)


In [71]:
empty_nodes_time = gpd.read_file(test_dir + "empty_nodes_time_farmacia.gpkg")
# Show
print(empty_nodes_time.shape)
empty_nodes_time.head(1)

(43734, 5)


,osmid,x,y,street_count,geometry
0,386138,-70.646254,-33.442901,4,POINT (-70.64625 -33.44290)


##### Problem 2 was that merge and column assignment was being done at the same time. Solution: Split in merge, then rename col.

In [92]:
# Problem 2 was that merge and column assignment was being done at the same time.
test = empty_nodes_time.copy()
poi_name = 'farmacia'
batch_time_col = 'time_'+str(0)+poi_name 

test = pd.merge(test,farmacias_batch0[['osmid','dist_'+poi_name]],on='osmid')
test.rename(columns={f"dist_{poi_name}":'time_'+str(0)+poi_name},inplace=True)
test.loc[test.osmid==farmacia_osmid]

,osmid,x,y,street_count,geometry,time_0farmacia
29587,6067607409,-70.712556,-33.457744,3,POINT (-70.71256 -33.45774),6.958944


In [75]:
farmacias_batch0 = gpd.read_file(test_dir + "03_splitting_merge_rename_col/nodes_distance_prep_farmacia_250batch0.gpkg")
farmacias_batch1 = gpd.read_file(test_dir + "03_splitting_merge_rename_col/nodes_distance_prep_farmacia_250batch1.gpkg")
farmacias_batch2 = gpd.read_file(test_dir + "03_splitting_merge_rename_col/nodes_distance_prep_farmacia_250batch2.gpkg")
farmacias_batch3 = gpd.read_file(test_dir + "03_splitting_merge_rename_col/nodes_distance_prep_farmacia_250batch3.gpkg")

In [76]:
farmacias_batch0.loc[farmacias_batch0.osmid==farmacia_osmid]

,osmid,index,x,y,street_count,dist_farmacia,farmacia_15min,geometry
29587,6067607409,29696,-70.712556,-33.457744,3,6.958944,3,POINT (-70.71256 -33.45774)


In [77]:
farmacias_batch1.loc[farmacias_batch1.osmid==farmacia_osmid]

,osmid,index,x,y,street_count,dist_farmacia,farmacia_15min,geometry
29587,6067607409,29696,-70.712556,-33.457744,3,14.6326,1,POINT (-70.71256 -33.45774)


In [78]:
farmacias_batch2.loc[farmacias_batch2.osmid==farmacia_osmid]

,osmid,index,x,y,street_count,dist_farmacia,farmacia_15min,geometry
29587,6067607409,29696,-70.712556,-33.457744,3,0.0,3,POINT (-70.71256 -33.45774)


In [79]:
farmacias_batch3.loc[farmacias_batch3.osmid==farmacia_osmid]

,osmid,index,x,y,street_count,dist_farmacia,farmacia_15min,geometry
29587,6067607409,29696,-70.712556,-33.457744,3,0.0,1,POINT (-70.71256 -33.45774)


In [80]:
farmacias_allbatches = gpd.read_file(test_dir + "03_splitting_merge_rename_col/nodes_time_farmacia.gpkg")
farmacias_allbatches.loc[farmacias_allbatches.osmid==farmacia_osmid]

,osmid,x,y,street_count,time_0farmacia,farmacia_0_15min,time_1farmacia,farmacia_1_15min,time_2farmacia,farmacia_2_15min,time_3farmacia,farmacia_3_15min,time_farmacia,farmacia_15min,geometry
29587,6067607409,-70.712556,-33.457744,3,6.958944,3,14.6326,1,0.0,3,0.0,1,0.0,8,POINT (-70.71256 -33.45774)


#### __Final test:__ Is the result correctly saved __(0.0 in time, 8 in count)__ in the project's (Script27b) nodes_analysis? (Success)

In [94]:
p_code = '00'
nodes_analysis_project = gpd.read_file(accesibilidad_urbana+f"data/external/santiago/output/project_{p_code}_osmnx/santiago_nodesanalysis_project_{p_code}.gpkg")
nodes_analysis_project.loc[nodes_analysis_project.osmid==farmacia_osmid]

,osmid,farmacia_time,farmacia_count_15min,geometry
29587,6067607409,0.0,8,POINT (-70.71256 -33.45774)


## Script 27b debugging - Project's data only available in certain zones.

#### * __Problem:__ When running not only farmacias, but other amenities, data is not available in a lot of zones. (NULLs, 0s).
#### * __Approach:__ Run farmacias only. Analyse pois_time() test outputs and Script 27b test outputs untill nodes_analysis.
#### * __Solution:__ Merge needed to be __<how='left'>__. Fixed.